In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import date

In [2]:
data = pd.read_csv("data/df_new.csv")

In [3]:
data.Gender = pd.get_dummies(data.Gender, columns = ['Gender'], drop_first = True)
data = pd.merge(data, pd.get_dummies(data.Neighbourhood, columns = ['Neighbourhood'], drop_first = True), left_index=True, right_index=True)
#data = data.drop(columns=['Neighbourhood'])

### 1. Split data for optim part

In [4]:
df_opti = data[data.ScheduledTime >= "2016-05-30"]
df_opti = df_opti[df_opti.ScheduledTime <= "2016-06-05"]

df_opti.to_csv("data/data_opti.csv", index = False)

In [5]:
df_pred = data[data.ScheduledTime < "2016-05-30"]

In [6]:
df_train = df_pred[df_pred.AppointmentTime < "2016-05-30"]

In [19]:
df_train.groupby("Neighbourhood").count().iloc[30:50,:]

,PatientId,AppointmentID,Gender,ScheduledTime,AppointmentTime,Age,Scholarship,Hipertension,Diabetes,Alcoholism,...,SANTOS REIS,SEGURANÇA DO LAR,SOLON BORGES,SÃO BENEDITO,SÃO CRISTÓVÃO,SÃO JOSÉ,SÃO PEDRO,TABUAZEIRO,UNIVERSITÁRIO,VILA RUBIM
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
ILHA DE SANTA MARIA,1261,1261,1261,1261,1261,1261,1261,1261,1261,1261,...,1261,1261,1261,1261,1261,1261,1261,1261,1261,1261
ILHA DO BOI,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
ILHA DO FRADE,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
ILHA DO PRÍNCIPE,1535,1535,1535,1535,1535,1535,1535,1535,1535,1535,...,1535,1535,1535,1535,1535,1535,1535,1535,1535,1535
ILHAS OCEÂNICAS DE TRINDADE,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
INHANGUETÁ,719,719,719,719,719,719,719,719,719,719,...,719,719,719,719,719,719,719,719,719,719
ITARARÉ,2503,2503,2503,2503,2503,2503,2503,2503,2503,2503,...,2503,2503,2503,2503,2503,2503,2503,2503,2503,2503
JABOUR,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749,...,1749,1749,1749,1749,1749,1749,1749,1749,1749,1749
JARDIM CAMBURI,5171,5171,5171,5171,5171,5171,5171,5171,5171,5171,...,5171,5171,5171,5171,5171,5171,5171,5171,5171,5171


df_train.to_csv("data/data_train.csv", index= False)

df_pred.to_csv("data/data_pred.csv", index = False)

### 2. Select one clinic and nb doctors

Select clinic with enough appointments and high no show rate : 5800 appointments and 21% no show for Maria Ortiz

In [115]:
#neigh = "MARUÍPE"
neigh = "MARIA ORTIZ"

n_doc = np.floor(len(df_opti[df_opti.Neighbourhood == neigh])/(5*10))

In [116]:
n_doc = 22

In [117]:
df_pred_c = df_pred[df_pred[neigh] == 1]

In [118]:
len(df_pred_c)

4887

In [119]:
np.mean(df_pred_c.NoShow)

0.23204419889502761

In [120]:
df_opti_c = df_opti[df_opti[neigh] == 1]

In [121]:
len(df_opti_c)

635

In [122]:
np.mean(df_opti_c.NoShow)

0.08818897637795275

### 3. Get prebooked slots

In [100]:
df_pred_c = df_pred_c[df_pred_c.AppointmentTime >= "2016-05-30"]
df_pred_c = df_pred_c[df_pred_c.AppointmentTime <= "2016-06-05"]
df_pred_c['AppointmentTime']= pd.to_datetime(df_pred_c['AppointmentTime'])
df_pred_c.AppointmentTime = df_pred_c.AppointmentTime.dt.tz_localize(None)
df_pred_c["slot"] = df_pred_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)
#df_pred_c.to_csv("data/pred_Maria.csv")
df_pred_c.to_csv("data/pred_Maruipe.csv")

In [101]:
df_opti_c = df_opti_c[df_opti_c.AppointmentTime >= "2016-05-30"]
df_opti_c = df_opti_c[df_opti_c.AppointmentTime <= "2016-06-05"]
#df_opti_c.to_csv("data/opti_Maria.csv")

In [102]:
df_opti_c

,PatientId,AppointmentID,Gender,ScheduledTime,AppointmentTime,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,SANTOS REIS,SEGURANÇA DO LAR,SOLON BORGES,SÃO BENEDITO,SÃO CRISTÓVÃO,SÃO JOSÉ,SÃO PEDRO,TABUAZEIRO,UNIVERSITÁRIO,VILA RUBIM
6940,22867,91845,0,2016-05-30 09:33:16+00:00,2016-05-30 23:59:59+00:00,47,MARUÍPE,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8596,27489,91088,0,2016-05-30 07:58:05+00:00,2016-05-30 23:59:59+00:00,50,MARUÍPE,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9407,34779,92429,0,2016-05-30 11:30:27+00:00,2016-05-31 23:59:59+00:00,64,MARUÍPE,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9409,35643,92440,0,2016-05-30 11:34:19+00:00,2016-05-31 23:59:59+00:00,64,MARUÍPE,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9411,748,96344,0,2016-05-31 14:40:06+00:00,2016-05-31 23:59:59+00:00,41,MARUÍPE,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108505,32297,104017,0,2016-06-03 11:33:42+00:00,2016-06-03 23:59:59+00:00,69,MARUÍPE,0,1,1,...,0,0,0,0,0,0,0,0,0,0
108746,31592,96251,1,2016-05-31 14:10:29+00:00,2016-06-02 23:59:59+00:00,55,MARUÍPE,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109292,27602,95803,1,2016-05-31 11:52:34+00:00,2016-06-02 23:59:59+00:00,74,MARUÍPE,0,1,0,...,0,0,0,0,0,0,0,0,0,0
109668,40304,93688,0,2016-05-30 16:26:53+00:00,2016-06-01 23:59:59+00:00,3,MARUÍPE,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Generate random availability patient

In [103]:
for i in range(10):
    df_opti_c.loc[:,"Availability_"+str(i)] = [random.randint(0,49) for n in range(len(df_opti_c))]

df_opti_c.to_csv("data/opti_Maria.csv")

### 5. Generate day availability to compare models

In [104]:
df_opti_c['AppointmentTime']= pd.to_datetime(df_opti_c['AppointmentTime'])
df_opti_c.AppointmentTime = df_opti_c.AppointmentTime.dt.tz_localize(None)
for i in range(5):
    df_opti_c.loc[:,"Availability_day_"+str(i)] = df_opti_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)

In [105]:
max(df_opti_c.Availability_day_0)

49

df_opti_c.to_csv("data/opti_Maria.csv")

In [106]:
df_opti_c.to_csv("data/opti_Maruipe.csv")

### 6. Generate data for predictions (week)

In [107]:
df_opti_c.AppointmentYear.unique()

array([2016], dtype=int64)

In [108]:
map_matrix = np.zeros((50,3)) #day, month, weekday

In [109]:
map_matrix[:,0] = [30]*10 + [31]*10 + [1]*10 + [2]*10 + [3]*10
map_matrix[:,1] = [5]*20 + [6]*30
map_matrix[:,2] = [0]*10 + [1]*10 + [2]*10 + [3]*10 + [4]*10

In [110]:
newdf = pd.DataFrame(np.repeat(df_opti_c.values[:,:119], 10, axis=0))
newdf.columns = df_opti_c.columns[:119]
newdf["Hour_available"] = df_opti_c.iloc[:,109:119].to_numpy().flatten()

In [111]:
newdf["AppointmentDay"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,0]))
newdf["AppointmentMonth"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,1]))
newdf["AppointmentWeekDay"] = newdf["Hour_available"].map(lambda x: int(map_matrix[x,2]))
newdf["AwaitingTime"] = (newdf.AppointmentDay - newdf.ScheduledDay)%31

In [112]:
newdf.to_csv("data/df_new_week_maruipe.csv", index = False)